In [24]:
import numpy as np
from dca import synth_data
from dca.dca import DynamicalComponentsAnalysis as DCA

from dca_research import pf_dca
from imp import reload

In [46]:
X = synth_data.gen_lorenz_data(100000)

In [48]:
model1 = DCA(T=3, d=1, verbose=True, n_init=5)
model1.fit(X)

PI: 7.6074 nats, reg: 0.0
PI: 7.7635 nats, reg: 0.0171
PI: 7.8359 nats, reg: 0.0471
PI: 7.9858 nats, reg: 0.0389
PI: 8.0989 nats, reg: 0.0082
PI: 8.2 nats, reg: 0.0029
PI: 8.2301 nats, reg: 0.0259
PI: 8.2642 nats, reg: 0.0412
PI: 8.3556 nats, reg: 0.0637
PI: 8.4973 nats, reg: 0.0194
PI: 8.5584 nats, reg: 0.0355
PI: 8.7113 nats, reg: 0.064
PI: 8.9451 nats, reg: 0.1264
PI: 9.068 nats, reg: 0.0956
PI: 9.1257 nats, reg: 0.0136
PI: 9.1614 nats, reg: 0.0004
PI: 9.1672 nats, reg: 0.0
PI: 9.171 nats, reg: 0.0
PI: 9.1716 nats, reg: 0.0
PI: 9.1716 nats, reg: 0.0
PI: 8.0141 nats, reg: 0.0
PI: 8.0948 nats, reg: 0.0123
PI: 8.1238 nats, reg: 0.0293
PI: 8.2014 nats, reg: 0.0168
PI: 8.2783 nats, reg: 0.0006
PI: 8.316 nats, reg: 0.0129
PI: 8.3598 nats, reg: 0.0229
PI: 8.4548 nats, reg: 0.075
PI: 8.627 nats, reg: 0.0071
PI: 8.7116 nats, reg: 0.0395
PI: 8.7905 nats, reg: 0.0916
PI: 8.9381 nats, reg: 0.1531
PI: 9.1178 nats, reg: 0.0012
PI: 9.1406 nats, reg: 0.0094
PI: 9.1715 nats, reg: 0.0
PI: 9.1716 nats

In [52]:
print(model1.coef_)
print(model2.coef_[0])
print(model2.coef_[1])

[[-0.99813071]
 [ 0.05349373]
 [ 0.02955512]]
[[-0.01796673]
 [-0.83400459]
 [-0.5514649 ]]
[[ 0.15986912]
 [-0.9870649 ]
 [ 0.01203101]]


In [55]:
print(model1.coef_.T.dot(model2.coef_[0]))

[[-0.04297948]]


In [56]:
print(model1.coef_.T.dot(model2.coef_[1]))

[[-0.21201649]]


In [62]:
reload(pf_dca)
model2 = pf_dca.PastFutureDynamicalComponentsAnalysis(T=3, d=1, verbose=True)
model2.fit(X)

PI: -0.9644 nats, reg: 0.0
PI: -0.5348 nats, reg: 0.0637
PI: -0.513 nats, reg: 0.0438
PI: -0.4556 nats, reg: 0.0019
PI: -0.4491 nats, reg: 0.0031
PI: -0.3833 nats, reg: 0.0221
PI: 0.7918 nats, reg: 0.1482
PI: 3.8952 nats, reg: 0.138
PI: 6.1619 nats, reg: 0.138
